In [0]:
source = 'OSCE'
xml_veraion = '2.0'

In [0]:
import json,os
import pyspark.sql.functions as F
from pyspark.sql.types import StructField, StructType, StringType, ArrayType, LongType, TimestampType, BinaryType, IntegerType, DateType
from datetime import datetime, timedelta
import re
from pyspark.sql import Window

In [0]:
start_date= (datetime.now() + timedelta(days = -30)).strftime("%Y-%m-%d")
end_date = datetime.now().strftime("%Y-%m-%d")
print("start_date:",start_date)
print("end_date:",end_date)

start_date: 2022-03-19
end_date: 2022-04-18

In [0]:
%fs ls /mnt/bifconnect/poc_folder

path,name,size,modificationTime
dbfs:/mnt/bifconnect/poc_folder/2022-04-07/,2022-04-07/,0,0
dbfs:/mnt/bifconnect/poc_folder/2022-04-08/,2022-04-08/,0,0
dbfs:/mnt/bifconnect/poc_folder/2022-04-13/,2022-04-13/,0,0
dbfs:/mnt/bifconnect/poc_folder/TBL_DATA_RAW/,TBL_DATA_RAW/,0,0
dbfs:/mnt/bifconnect/poc_folder/TBL_DATA_RAW.json,TBL_DATA_RAW.json,66499954,1647598296000
dbfs:/mnt/bifconnect/poc_folder/tempDirs/,tempDirs/,0,0
dbfs:/mnt/bifconnect/poc_folder/xml/,xml/,0,0


In [0]:
root = '/dbfs/mnt/bifconnect/poc_folder/xml/OSCE/20220101/'
f_list = [] 
for fname in os.listdir(root):
    f_dict = {}
    f_dict['file_name'] = fname
    f_dict['file_path'] = root + fname
    f_list.append(f_dict)

In [0]:
def read_xml_file(path):
    print(path)
    with open(path, 'r' ) as f:
        xml_str = f.read()
    return xml_str[1:]

read_xml_file_udf = udf(read_xml_file, StringType())

file_info_schema = StructType([
    StructField('file_date', DateType(), True),
    StructField('xml_version', StringType(), True),
    StructField('ip', StringType(), True)
  
])

def file_name_split(path):
    file_info = {}
    list = path.split('_')
    file_info['file_date'] =  datetime.strptime(list[1], "%Y%m%d%H%M%S%f")
    file_info['xml_version'] = list[2]
    file_info['ip'] = list[3]
    return file_info

file_name_split_udf = udf(file_name_split, file_info_schema)

  

In [0]:
file_rdd = spark.sparkContext.parallelize(f_list)
df_raw = (spark.createDataFrame(file_rdd)
          .select('file_name', read_xml_file_udf('file_path').alias('file_content'))
          .withColumn('file_info',file_name_split_udf('file_name'))
          .select('file_info.*','file_name','file_content')
          .withColumn('source', F.lit('OSCE'))
          )
display(df_raw)
